# Hello, Chameleon

In this notebook, you will pick up where you left off after creating a Chameleon account, joining a Chameleon project, and preparing key pair. Now, you will learn how to:

-   Reserve resources in Chameleon
-   Access your reserved resources over SSH
-   Execute commands on your resources
-   Retrieving files saved on Chameleon resources
-   Extend your Chameleon lease (in case you need more time) or delete it (in case you finish early)

## Exercise: reserve resources

Whenever you run an experiment on Chameleon, you will

1.  Open a Python notebook, which includes commands to reserve the resources (VMs, bare metal servers, or networks) that you need for your experiment. Run these commands.
2.  Wait until the resources in your experiment are ready to log in.
3.  Log in to the resources and run your experiment (either by executing commands in the notebook, or by using SSH in a terminal and running commands in those SSH sessions).

Also, when you finish an experiment and have saved all the data somewhere safe, you will *delete* the resources in your experiment to free them for use by other experimenters.

In this exercise, we will reserve a resource on Chameleon.

### Generating a Virtual Machine on chameleon

Run this cell to initialize the environment, also make sure change the vriable “project_name”. project_name looks like “CHI-XXXXX” and it is the name of your project which you were assigned.

In [ ]:
import chi,os
project_name = "CHI-XXXXXX"
os.environ["OS_PROJECT_NAME"] = project_name
chi.set("project_name", project_name)  
chi.use_site("KVM@TACC")

To ensure uniqueness, each server within a project must have a distinct name. To differentiate your servers from those of your peers, the server’s name should be composed of your chameleon username and an exp_name that your instructor has specified in the cell provided below.

In [ ]:
exp_name = ""
user = os.getenv("USER")
server_name = f"{exp_name}_{user}"

### Creating the server

In [ ]:
import chi.server
flavor = "m1.small"
image_name = "CC-Ubuntu20.04"
server = chi.server.create_server(server_name, 
                                  image_name=image_name, 
                                  flavor_name=flavor)

server_id = server.id
chi.server.wait_for_active(server_id)

Associate an IP address with this server:

In [ ]:
reserved_fip = chi.server.associate_floating_ip(server_id)
reserved_fip

### Creating a Security group

A security group named “Allow SSH” will be generated in the following cell for our project, enabling us to connect to the remote server from our local desktop.

In [ ]:
%%bash
export OS_AUTH_URL=https://kvm.tacc.chameleoncloud.org:5000/v3
export OS_REGION_NAME="KVM@TACC"

access_token=$(curl -s -H"authorization: token $JUPYTERHUB_API_TOKEN"     "$JUPYTERHUB_API_URL/users/$JUPYTERHUB_USER"     | jq -r .auth_state.access_token)
export OS_ACCESS_TOKEN="$access_token"
SECURITY_GROUP_NAME="Allow SSH"

if ! openstack security group show "$SECURITY_GROUP_NAME" > /dev/null 2>&1; then
    openstack security group create "$SECURITY_GROUP_NAME"  --description "Enable SSH traffic on TCP port 22"
    openstack security group rule create "$SECURITY_GROUP_NAME" \
     --protocol tcp --dst-port 22:22 --remote-ip 0.0.0.0/0


else
    echo "Security group already exists"
fi

The preceding cell generated a security group, and this cell will attach that security group to our server, making it ready to be accessed via SSH.

In [ ]:
nova_server = chi.nova().servers.get(server_id)
nova_server.add_security_group("Allow SSH")
f"updated security groups: {[group.name for group in nova_server.list_security_group()]}"

Wait for the server to be ready to connect.

In [ ]:
server.wait_for_tcp(reserved_fip, port=22)

Now our resources are reserved and ready to login through SSH

## Exercise: log in to resources and execute commands

### Logging in over SSH via local terminal

Once your server is ready to use. you can follow the guidelines below to log in to your server via SSH.

Run the cell below and use it’s output as the exact command to login through your laptop’s terminal.

In [ ]:

print(f"ssh -i ~/.ssh/id_rsa_chameleon cc@{reserved_fip}")


The first time you log in to each new host, your computer will display a warning similar to the following:

``` shell
The authenticity of host "129.114.26.xx (129.114.26.xx)" cannot be established.
ED25519 key fingerprint is SHA256:1fcbGrgLDdOeorauhz3CTyhmFqOHsrEWlu0TZ6yGoDM.
This key is not known by any other names
Are you sure you want to continue connecting (yes/no/[fingerprint])?
```

and you will have to type the word *yes* and hit Enter to continue. If you have specified your key path and other details correctly, it won’t ask you for a password when you log in to the node. (It may ask for the passphrase for your private key if you’ve set one.)

The output of the above command will look somewhat like this.

``` shell
Welcome to Ubuntu 20.04.4 LTS (GNU/Linux 5.4.0-124-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Thu Feb 23 17:52:13 UTC 2023

  System load:  0.08               Processes:             143
  Usage of /:   10.5% of 36.90GB   Users logged in:       1
  Memory usage: 12%                IPv4 address for ens3: 10.56.0.154
  Swap usage:   0%


0 updates can be applied immediately.


Last login: Thu Feb 23 16:44:05 2023 from 100.35.242.215
cc@cp3793-nyu-edu-fount:~$
```

We will create a file hello.txt on our remote machine.

``` shell
:~$ echo "Hello from $(hostname)" > hello.txt
```

Now we will use this file “hello.txt” in the later exercises where we will see how transfering of file works between remote host and local host.

## Exercise: transfer files to and from resources

While working on a remote host we have to transfer files from remote to local and vice versa. To move data back and forth between your laptop and remote system that you access with *ssh*, we can use *scp*. The syntax is:

``` shell
scp [OPTIONS] SOURCE DESTINATION
```

where SOURCE is the full address of the location where the file is currently llocated, and DESTINATION is the address of the location that you want to copy a file to.

When you are transferring a file from a remote host to your laptop, you will run scp from a terminal on your laptop (NOT a terminal that is logged in to the remote host).

### Transfering files through the local terminal

Upon accessing the remote host via our local environment’s terminal, we generated a file named “hello.txt”. In this exercise, we will execute an “scp” command to transfer the file from the remote host to our laptop.

``` shell
user@username:~$ scp -i ~/.ssh/id_rsa_chameleon cc@reserved_fip:/home/cc/hello.txt .
hello.txt                       100%    1KB     0.1KB/s   00:00
user@username:~$
```

Run the code below and you will get the exact command which you have to use in your local terminal.

In [ ]:
print(f'scp -i ~/.ssh/id_rsa_chameleon cc@{reserved_fip}:/home/cc/hello.txt .')


We have successfully transferred “hello.txt” from the remote host to our laptop. We can now open the file in any text editor, make changes as necessary, and then attempt to transfer the updated file back to the remote host.

``` shell
user@username:~$ scp hello.txt cc@reserved_fip:/home/cc/chameleon/
hello.txt                       100%    1KB     0.1KB/s   00:00
user@username:~$
```

Run the code below and you will get the exact command which you have to use in your local terminal to transfer the file back to remote host.

In [ ]:
print(f'scp -i ~/.ssh/id_rsa_chameleon hello.txt cc@{reserved_fip}:/home/cc/')


Use of `-i ~/.ssh/id_rsa_chameleon` is optional if your Chameleon key is in the default location.

## Exercise: delete resources

After utilizing the resources, you can remove them by updating the DELETE parameter to True and executing the cell below.

Note that removing the resources will revoke your access to them, and all the information stored on them will be erased. Therefore, ensure that you have saved all your work before deleting the resources.

In [ ]:
DELETE = False

if DELETE:
    chi.server.delete_server(server_id)
    ip_details = chi.network.get_floating_ip(reserved_fip)
    chi.neutron().delete_floatingip(ip_details["id"])